In [ ]:
conda create -n simpletransformers python pandas tqdm -y

In [ ]:
conda activate simpletransformers

In [ ]:
conda install pytorch cpuonly -c pytorch

In [ ]:
!pip install -U transformers==4.7.0 datasets scipy scikit-learn

In [ ]:
import matplotlib.pyplot as plt
import random
import logging
from IPython.display import display, HTML

import numpy as np
import pandas as pd
import datasets
from datasets import Dataset, load_dataset, load_metric, ClassLabel, Sequence
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.model_selection import train_test_split

from sklearn.metrics import precision_recall_fscore_support

# TRAIN_FILE_PATH = '/content/drive/MyDrive/mini_project/english_news/English_news_data/en_news_with_health.csv'
TRAIN_FILE_PATH = '/kaggle/input/english-data/en_final_dataset.csv'  # 정치 데이터


In [ ]:
data = pd.read_csv(TRAIN_FILE_PATH)

data = data.rename(columns={'topic_idx':'label'})
data['label'] = data['label'].astype('int')

data_dummies = pd.get_dummies(data, columns=['label'])

data_dummies.head()

In [ ]:
data = data_dummies.drop(['Unnamed: 0', 'title_len'], axis=1)
data

In [ ]:
# 사이트 
data['labels'] = list(zip(data.label_0.tolist(), data.label_1.tolist(), data.label_2.tolist(), data.label_3.tolist(),  data.label_4.tolist(), data.label_5.tolist(), data.label_6.tolist()))
data['text'] = data['title'].apply(lambda x: x.replace('\n', ' '))

data.head()

In [ ]:
# 사이트

from sklearn.model_selection import train_test_split


train_df, eval_df = train_test_split(data, test_size=0.2)

In [ ]:
# train_df.head()

In [ ]:
!pip install simpletransformers

In [ ]:
# 사이트
from simpletransformers.classification import MultiLabelClassificationModel

model = MultiLabelClassificationModel('roberta', 'roberta-base', num_labels=7, args={'train_batch_size':2, 'gradient_accumulation_steps':16, 'learning_rate': 3e-5, 'num_train_epochs': 3, 'max_seq_length': 100})

In [ ]:
# self.args = {
#     "output_dir": "outputs/",
#     "cache_dir": "cache_dir/",

#     "fp16": True,
#     "fp16_opt_level": "O1",
#     "max_seq_length": 128,
#     "train_batch_size": 8,
#     "gradient_accumulation_steps": 1,
#     "eval_batch_size": 8,
#     "num_train_epochs": 1,
#     "weight_decay": 0,
#     "learning_rate": 4e-5,
#     "adam_epsilon": 1e-8,
#     "warmup_ratio": 0.06,
#     "warmup_steps": 0,
#     "max_grad_norm": 1.0,

#     "logging_steps": 50,
#     "save_steps": 2000,

#     "overwrite_output_dir": False,
#     "reprocess_input_data": False,
#     "evaluate_during_training": False,

#     "process_count": cpu_count() - 2 if cpu_count() > 2 else 1,
#     "n_gpu": 1,
# }

In [ ]:
model.train_model(train_df, error_if_nonfinite=False)

In [ ]:
result, model_outputs, wrong_predictions = model.eval_model(eval_df)

In [ ]:
result, model_outputs # 정확도, eval_loss 출력

In [ ]:
predictions, raw_outputs = model.predict(["Sam was a Wizard"]) # 테스트용

In [ ]:
predictions, raw_outputs